In [1]:
{-# LANGUAGE OverloadedStrings #-}

import AdventOfCode
import qualified Data.Graph.Inductive as G
import qualified Data.Attoparsec.ByteString.Char8 as A

Line 1: Unused LANGUAGE pragma
Found:
{-# LANGUAGE OverloadedStrings #-}
Why not:

In [2]:
input <- dayLines 7

In [3]:
import Data.Functor (($>))

parseBag = do
    adjective <- tillSpace
    A.skipSpace
    colour <- tillSpace
    A.skipSpace
    A.choice [A.string "bags", A.string "bag"]
    pure (adjective <> " " <> colour)

numberedBag = do
    n <- A.decimal
    A.skipSpace
    bag <- parseBag
    pure (n, bag)

parseLine = do
    bag <- parseBag
    A.skipSpace
    A.string "contain"
    A.skipSpace
    bags <- A.choice
        [ A.string "no other bags" $> []
        , numberedBag `A.sepBy` A.string ", "
        ]
    A.char '.'
    pure (bag, bags)

In [4]:
parsedInput = map (parsed parseLine) input

In [5]:
import qualified Data.Map.Strict as M
luggageMap = M.fromList parsedInput
colours = M.keys luggageMap
numberedColours = M.fromList $ zip colours [0..]

numberedLuggageMap = M.map (map $ \(n, bagType) -> (n, numberedColours M.! bagType))
    $ M.mapKeys (numberedColours M.!) luggageMap

edges = concatMap (\(bagType, numberedBagTypes) -> zip (repeat bagType) numberedBagTypes)
    $ M.toList numberedLuggageMap
    

massagedNodes = map (\(name, idNo) -> (idNo, name)) $ M.toList numberedColours
massagedEdges = map (\(nodeA, (quantity, nodeB)) -> (nodeA, nodeB, quantity)) edges

graph = G.mkGraph massagedNodes massagedEdges

Line 6: Use second
Found:
\ (n, bagType) -> (n, numberedColours M.! bagType)
Why not:
Data.Bifunctor.second ((M.!) numberedColours)

In [6]:
import Data.Graph.Inductive.PatriciaTree (Gr)
import qualified Data.ByteString as BS
import Data.List

length $ G.reachable (numberedColours M.! "shiny gold") (G.grev graph :: Gr BS.ByteString Int) \\ [numberedColours M.! "shiny gold"]

248

In [7]:
totalBags :: M.Map BS.ByteString [(Int, BS.ByteString)] -> BS.ByteString -> Int
totalBags bagMap colour = let
    insides = bagMap M.! colour
    total = sum $ map (\(n,c) -> n * totalBags bagMap c) insides
    in total+1

In [8]:
totalBags luggageMap "shiny gold" - 1

57281